In [1]:
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
import numpy as np
import re

tokenizer = Tokenizer()

In [2]:
def vowels_of(kana):
    table = {
        'ア':'a','イ':'i','ウ':'u','エ':'e','オ':'o',
        'ャ':'a','ュ':'u','ョ':'o',
        'カ':'a','キ':'i','ク':'u','ケ':'e','コ':'o',
        'サ':'a','シ':'i','ス':'u','セ':'e','ソ':'o',
        'タ':'a','チ':'i','ツ':'u','テ':'e','ト':'o',
        'ナ':'a','ニ':'i','ヌ':'u','ネ':'e','ノ':'o',
        'ハ':'a','ヒ':'i','フ':'u','ヘ':'e','ホ':'o',
        'マ':'a','ミ':'i','ム':'u','メ':'e','モ':'o',
        'ヤ':'a','ユ':'u','ヨ':'o',
        'ラ':'a','リ':'i','ル':'u','レ':'e','ロ':'o',
        'ワ':'a','ヲ':'o','ン':'n',
        'ガ':'a','ギ':'i','グ':'u','ゲ':'e','ゴ':'o',
        'ザ':'a','ジ':'i','ズ':'u','ゼ':'e','ゾ':'o',
        'ダ':'a','ヂ':'i','ヅ':'u','デ':'e','ド':'o',
        'バ':'a','ビ':'i','ブ':'u','ベ':'e','ボ':'o',
        'パ':'a','ピ':'i','プ':'u','ペ':'e','ポ':'o',
        'ー':'', 'ッ':''
    }
    return "".join(table.get(c, "") for c in kana)

In [3]:
def consonants_of(kana):
    cons_map = {
        'ア':'','イ':'','ウ':'','エ':'','オ':'',
        'カ':'k','キ':'k','ク':'k','ケ':'k','コ':'k',
        'サ':'s','シ':'s','ス':'s','セ':'s','ソ':'s',
        'タ':'t','チ':'t','ツ':'t','テ':'t','ト':'t',
        'ナ':'n','ニ':'n','ヌ':'n','ネ':'n','ノ':'n',
        'ハ':'h','ヒ':'h','フ':'h','ヘ':'h','ホ':'h',
        'マ':'m','ミ':'m','ム':'m','メ':'m','モ':'m',
        'ヤ':'y','ユ':'y','ヨ':'y',
        'ラ':'r','リ':'r','ル':'r','レ':'r','ロ':'r',
        'ワ':'w','ヲ':'w','ン':'n',
        'ガ':'g','ギ':'g','グ':'g','ゲ':'g','ゴ':'g',
        'ザ':'z','ジ':'z','ズ':'z','ゼ':'z','ゾ':'z',
        'ダ':'d','ヂ':'d','ヅ':'d','デ':'d','ド':'d',
        'バ':'b','ビ':'b','ブ':'b','ベ':'b','ボ':'b',
        'パ':'p','ピ':'p','プ':'p','ペ':'p','ポ':'p',
        'ー':'','ッ':'t',
    }
    return "".join(cons_map.get(c, "") for c in kana)

In [4]:
def levenshtein(a, b):
    if not a or not b: return max(len(a), len(b))
    dp = [[i + j if i * j == 0 else 0 for j in range(len(b) + 1)] for i in range(len(a) + 1)]
    for i in range(1, len(a) + 1):
        for j in range(1, len(b) + 1):
            dp[i][j] = min(
                dp[i-1][j] + 1,
                dp[i][j-1] + 1,
                dp[i-1][j-1] + (0 if a[i-1] == b[j-1] else 1)
            )
    return dp[-1][-1]

def similarity(a, b):
    if not a or not b: return 0
    dist = levenshtein(a, b)
    return 1 - dist / max(len(a), len(b))

In [5]:
def extract_features(sentence):
    tokens = list(tokenizer.tokenize(sentence))
    readings = [t.reading for t in tokens if t.reading != "*"]

    features = {}

    # 語数
    features["word_count"] = len(tokens)

    # 末尾2語
    if len(readings) >= 2:
        r1 = readings[-1]
        r2 = readings[-2]

        # 読み末尾類似
        features["end_read_sim4"] = similarity(r1[-4:], r2[-4:])

        # 母音列類似
        v1 = vowels_of(r1)
        v2 = vowels_of(r2)
        features["vowel_sim"] = similarity(v1[-4:], v2[-4:])

        # 子音列類似
        c1 = consonants_of(r1)
        c2 = consonants_of(r2)
        features["cons_sim"] = similarity(c1[-4:], c2[-4:])
    else:
        features["end_read_sim4"] = 0
        features["vowel_sim"] = 0
        features["cons_sim"] = 0

    # 読み全体の長さ平均
    features["avg_reading_len"] = np.mean([len(r) for r in readings]) if readings else 0

    return features

In [6]:
jokes = [
    "布団がふっとんだ",
    "イクラはいくら？",
    "カレーはかれー",
    "トイレにいっといれ",
    "カエルが帰る",
    "ネコがねころんだ",
    "ラクダに乗るとらくだ",
    "カッターを買った。切れなかったー",
    "コーラをこおらせる",
    "チータが落っこちーた",
    "スキーが好き",
    "イルカはいるか？",
    "パンダのパンだ",
    "箱をはこぶ",
    "虫はむし",
    "栗のクリーム",
    "君は黄身が好き？",
    "ソーダはうまそうだ",
    "アイスを愛す",
    "ダジャレを言ったのは誰じゃ",
    "電話に、でんわ",
    "トマトが止まっとる",
    "タレがたれた",
    "スイカはやすいか？",
    "焼肉は焼きにくい",
    "廊下にすわろうか",
    "このタイヤ固いや",
    "内臓がないぞう",
    "ホットケーキはほっとけー",
    "鼻毛はなげー",
    "ワニが輪になった",
    "草がくさい"
]

labels = [
    2,2,1,2,2,1,2,2,1,2,1,2,1,2,2,2,2,2,2,1,
    1,2,1,2,1,2,1,1,2,1,  # 1〜30
    1,  # 31: ワニが輪になった
    2   # 32: 草がくさい
]

In [7]:
X = [extract_features(j) for j in jokes]
vec = DictVectorizer(sparse=False)
Xvec = vec.fit_transform(X)

model = LinearRegression()
model.fit(Xvec, labels)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [8]:
model.fit(Xvec, labels)
def judge(joke):
    f = extract_features(joke)
    Xtest = vec.transform([f])
    return model.predict(Xtest)[0]

def judge_text(score):
    if score >= 1.5:
        return "🤣 面白い！"
    else:
        return "😐 面白くないかも…"

# テスト
joke = "あわわわわわわわわわわわわわわわわわわわわわわわわわわわわ"
score = judge(joke)
print(joke, "→", score, judge_text(score))

あわわわわわわわわわわわわわわわわわわわわわわわわわわわわ → 2.2485067205381046 🤣 面白い！
